<a href="https://colab.research.google.com/github/minsaee/ai_chat_openAI/blob/master/007_LangchainChatbot_FAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chat_openAI'

Mounted at /content/drive
/content/drive/MyDrive/ai_chat_openAI


In [2]:
!pip install langchain openai==0.28.1 tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 여러 문서에서 찾아서 답변하는 챗봇 만들기

랭체인(langchain)이란?  
 - ChatGPT와 같은 대규모 언어 모델을 사용하여 어플리케이션을 쉽게 생성할 수 있도록 설계된 프레임워크이다.  
 - LLM에 외부의 '지식'이나 '계산능력'을 활용할 때 도움이 된다.  
 - 자신이 학습한 것만으로 대화하는 LLM에게 '책'이나 '프로그램'을 전달해서 외부의 '지식'이나 '계산능력'을 활용할 수 있게 하는 역할을 한다.

In [3]:
import os

os.environ["OPENAI_API_KEY"]='sk-'

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

### Load multiple and process documents

In [ ]:
# !wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
# !unzip data

In [5]:
# path : directory
# glob : file

loader = DirectoryLoader('./data/data_hwell', glob='*.txt', loader_cls=TextLoader)

documents = loader.load()

print('문서의 개수 :', len(documents))

문서의 개수 : 50


In [6]:
print('1번 문서 :', documents[1])
print('-'*20)
print('21번 문서:', documents[2])
print('-'*20)

1번 문서 : page_content='오십견이 걸렸을 경우 좋은 자가관리 방법은?\n통증이 장기간 지속되면 의사의 진찰 받는 것이 중요합니다. 그러나 오십견을 예방하고 또한 진단받은 후 치료의 일종으로 자가 관리 또한 중요합니다. 어깨통증 심할 경우에는 어깨를 너무 과도하게 사용하는 것과 심한 운동은 피하는 것이 좋습니다. 그리고, 의사의 치료를 통해 어는 정도 통증이 완화되면  어깨 운동이 필요합니다. 너무 과도한 운동은 오히려 해가 될 수 있으나 적절한 운동은 오십견에 많은 도움을 줄 수 있습니다.\n좋은 운동으로 관절의 가동범위(움직이는 범위)를 최대한 늘이는 것입니다. 다친 후에 움직이지 않으면 관절이 굳고, 굳는 것 때문에 통증이 더 올 수 있습니다. 처음에는 스스로 하지 말고 치료사나 타인에 의해서 관절운동을 수동적으로 하되 어깨관절의 앞, 뒤, 내측, 외측, 돌리기 등을 부드럽게 시행합니다(어깨관절의 굴곡, 신전, 외전, 내전, 외회전, 내회전). 점차적으로 약간씩 힘을 주어서 하며 괜찮은 경우 스스로 관절운동을 합니다. 어느정도 되면 저항을 주면서 운동을 합니다. 가동 범위도 처음에 20-30도 정도로 하다가 점차 늘려 나갑니다.\n관절운동시 통증이 계속 있을 수 있기 때문에 물리치료, 약물치료 등을 계속합니다. 만약에 관절이 굳어있으면 스트레칭을 시도합니다. 스트레칭은 주로 어깨관절 뒤쪽의 스트레칭에 중점을 두어 하며 반드시 아프지 않게 하여야 합니다.' metadata={'source': 'data/data_hwell/1.txt'}
--------------------
21번 문서: page_content='요통에 좋은 운동과 예방법은?\n요통의 예방과 치료는 무엇보다도 질병에 대한 교육이 중요하고 평소에 바른 자세 생활 등 자기 관리가 가장 중요하다고 하겠습니다. 과로에 대해서는 쉽게 피로하지 않는 건강한 몸을 만들어 주는 것이 중요하며, 다리와 허리를 튼튼하게 근 기능을 단련하는 것이 필요합니다. 요통의 예방에는 허리근육 강화가 아주 중

In [7]:
documents[1]

Document(page_content='오십견이 걸렸을 경우 좋은 자가관리 방법은?\n통증이 장기간 지속되면 의사의 진찰 받는 것이 중요합니다. 그러나 오십견을 예방하고 또한 진단받은 후 치료의 일종으로 자가 관리 또한 중요합니다. 어깨통증 심할 경우에는 어깨를 너무 과도하게 사용하는 것과 심한 운동은 피하는 것이 좋습니다. 그리고, 의사의 치료를 통해 어는 정도 통증이 완화되면  어깨 운동이 필요합니다. 너무 과도한 운동은 오히려 해가 될 수 있으나 적절한 운동은 오십견에 많은 도움을 줄 수 있습니다.\n좋은 운동으로 관절의 가동범위(움직이는 범위)를 최대한 늘이는 것입니다. 다친 후에 움직이지 않으면 관절이 굳고, 굳는 것 때문에 통증이 더 올 수 있습니다. 처음에는 스스로 하지 말고 치료사나 타인에 의해서 관절운동을 수동적으로 하되 어깨관절의 앞, 뒤, 내측, 외측, 돌리기 등을 부드럽게 시행합니다(어깨관절의 굴곡, 신전, 외전, 내전, 외회전, 내회전). 점차적으로 약간씩 힘을 주어서 하며 괜찮은 경우 스스로 관절운동을 합니다. 어느정도 되면 저항을 주면서 운동을 합니다. 가동 범위도 처음에 20-30도 정도로 하다가 점차 늘려 나갑니다.\n관절운동시 통증이 계속 있을 수 있기 때문에 물리치료, 약물치료 등을 계속합니다. 만약에 관절이 굳어있으면 스트레칭을 시도합니다. 스트레칭은 주로 어깨관절 뒤쪽의 스트레칭에 중점을 두어 하며 반드시 아프지 않게 하여야 합니다.', metadata={'source': 'data/data_hwell/1.txt'})

### Split texts(텍스트 분할하기)

- ChatGPT API의 GPT-3.5-turbo 모델은 입력으로 사용하는 텍스트와 ChatGPT API가 반환하는 답변의 길이를 모두 포함하여 최대 4,096 토큰을 처리할 수 있다.
- 예를 들어, ChatGPT API 입력으로 약 3,000토큰 정도의 길이를 가진 입력을 사용하면 ChatGPT는 최대 약 1,000토큰의 답변을 할 수 있는 구조이다.
- 입력의 길이가 4,096의 길이에 인접한 입력을 사용하면 ChatGPT API가 답변을 하던 도중 4,096개의 토큰을 모두 소진하면 더 이상 답변을 생성할 수 없어 텍스트가 중간에 끊길 수 있다. 또한 입력의 길이가 4,096을 넘으면 ChatGPT의 입력으로 사용할 수 없다는 에러가 발생한다.
- 이런 현상 때문에 너무 긴 문서들은 한 번에 입력으로 사용할 수 없으므로 여러 개의 뭉치(chunk)로 자른 후 ChatGPT API를 여러 번 호출하여 처리해야 한다.
- 랭체인의 RecursiveCharacterTextSplitter는 이를 위해 사용하는 도구로, 주어진 문서를 더 작은 단위의 텍스트 뭉치(chunk)로 잘라주는 역할을 한다.
-chunk_size에는 텍스트의 분할 기준이 되는 길이를 설정하고, chunk_overlap에는 텍스트를 분할할 때 텍스트의 내용을 얼마나 겹쳐서 자를 것인지 지정한다.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수:', len(texts))

분할된 텍스트의 개수: 61


In [9]:
source_list = []
for i in range(0, len(texts)):
  source_list.append(texts[i].metadata['source'])

element_counts = Counter(source_list)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서:', filtered_counts)
print('분할된 텍스트 개수:', len(documents)+len(filtered_counts))

2개 이상으로 분할된 문서: {'data/data_hwell/3.txt': 2, 'data/data_hwell/4.txt': 2, 'data/data_hwell/21.txt': 2, 'data/data_hwell/26.txt': 2, 'data/data_hwell/27.txt': 2, 'data/data_hwell/32.txt': 2, 'data/data_hwell/33.txt': 2, 'data/data_hwell/34.txt': 3, 'data/data_hwell/35.txt': 2, 'data/data_hwell/44.txt': 2}
분할된 텍스트 개수: 60


### Create Chroma DB

- ChromaDB는 임베딩과 코사인 유사도 계산, 텍스트 불러오는 과정들을 기능별로 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있게 도와주는 벡터응용 도구이다.
- Chroma.from_documents()함수를 이용해 벡터 도구 객체를 선언한다. 이때 documents에는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 전달하고, embedding에는 어떤 종류의 임베딩을 사용할 것인지를 기재한다.

In [10]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embedding
)

vectordb

### Make a retriever
 - 벡터 도구 객체를 선언하고 나면 as_retriever()함수를 이용해 입력된 텍스트로부터 유사한 텍스트를 찾아주는 검색기를 선언한다.

In [11]:
retriever = vectordb.as_retriever()

In [13]:
docs = retriever.get_relevant_documents('오십견 알려줘.')
print('유사 문서 개수:', len(docs))
print('--'*20)
print('첫번째 유사 문서:', docs[0])
print('--'*20)
print('각 유사 문서의 문서 출처:')
for doc in docs:
  print(doc.metadata["source"])

유사 문서 개수: 4
----------------------------------------
첫번째 유사 문서: page_content='오십견은 나이든 사람에게만 생기나요 ?\n아닙니다. 흔히 어깨가 아프면 오십견이라고 생각하기 쉬우나 어깨 통증의 원인은 오십견 뿐만 아니라 다음과 같이 많은 질환에 의해 야기될 수 있습니다.\n<어깨통증의 원인>\n1.오십견 (유착성 견관절낭염) 2.어깨 관절 충돌 증후군 (건초염, 점액낭염)\n3.회전근개 파열\n4.근막통 증후군\n5.관절염\n6.경추 디스크 질환\n7.경추 퇴행성 질환\n8.경추 신경근 병변\n9.내장 질환 (심근경색증, 폐 질환 등 )\n그중 오십견은 어깨관절의 한 구조물인 피낭의 염증으로 피낭을 섬유화하고 두껍게 하며, 활액의 반응도 유착되어 정상적인 피낭의 유연성이 소실되고, 외회전, 회전 및 내전의 가동력을 제한받는 것을 말합니다. 오십견은 다음과 같은 경우 발생될 수 있습니다.\n<오십견의 원인>\n1. 움직이지 않아서 발생\n2. 자가 면역 반응\n3. 부상 후 발생\n4. 점액낭염, 건초염, 충돌 증후군, 회전근개 파열과 동반\n이렇게 어깨구조물의 염증이 야기되고 최종적으로 어깨관절이 굳은 현상을 유착성 낭염이라고 합니다. 이것은 보통 오십대에 많이 발생해서 오십견으로 흔히 알려져 왔습니다. 그러나 오십견은 위와같은 원인이 있는 경우 연령에 상관없이 누구에게나 발생할 수 있습니다. 최근에는 운동부족과 장시간에 걸친 어깨의 과도한 사용으로 20-30대에서도 흔히 발생됩니다.\n오십견은 어깨의 유착성 관절낭염으로 어깨의 관절낭에 염증이 생기거나, 이와 동반되어 통증 및 굳어지는 증상이 생기는 질환입니다. 오십견이라 불리지만 반드시 오십대나 나이든 사람에게만 생기는 것은 아니며, 그보다 젊은 연령층에서도 발생할 수 있습니다. 팔을 많이 쓰는 사람이나 주부에게 많이 오는 것이 특징이며 최근에는 잘못된 자세로 컴퓨터를 장시간 사용하거나 스트레스, 운동부족 등도 원인이 되는 것으로 알려져 있습니다.'

### 결과를 k개 반환

In [14]:
retriever = vectordb.as_retriever(search_kwargs={'k':2})

In [15]:
docs = retriever.get_relevant_documents('오십견 알려줘.')
for doc in docs:
  print(doc.metadata['source'])
print(docs)

data/data_hwell/37.txt
data/data_hwell/39.txt
[Document(page_content='오십견은 나이든 사람에게만 생기나요 ?\n아닙니다. 흔히 어깨가 아프면 오십견이라고 생각하기 쉬우나 어깨 통증의 원인은 오십견 뿐만 아니라 다음과 같이 많은 질환에 의해 야기될 수 있습니다.\n<어깨통증의 원인>\n1.오십견 (유착성 견관절낭염) 2.어깨 관절 충돌 증후군 (건초염, 점액낭염)\n3.회전근개 파열\n4.근막통 증후군\n5.관절염\n6.경추 디스크 질환\n7.경추 퇴행성 질환\n8.경추 신경근 병변\n9.내장 질환 (심근경색증, 폐 질환 등 )\n그중 오십견은 어깨관절의 한 구조물인 피낭의 염증으로 피낭을 섬유화하고 두껍게 하며, 활액의 반응도 유착되어 정상적인 피낭의 유연성이 소실되고, 외회전, 회전 및 내전의 가동력을 제한받는 것을 말합니다. 오십견은 다음과 같은 경우 발생될 수 있습니다.\n<오십견의 원인>\n1. 움직이지 않아서 발생\n2. 자가 면역 반응\n3. 부상 후 발생\n4. 점액낭염, 건초염, 충돌 증후군, 회전근개 파열과 동반\n이렇게 어깨구조물의 염증이 야기되고 최종적으로 어깨관절이 굳은 현상을 유착성 낭염이라고 합니다. 이것은 보통 오십대에 많이 발생해서 오십견으로 흔히 알려져 왔습니다. 그러나 오십견은 위와같은 원인이 있는 경우 연령에 상관없이 누구에게나 발생할 수 있습니다. 최근에는 운동부족과 장시간에 걸친 어깨의 과도한 사용으로 20-30대에서도 흔히 발생됩니다.\n오십견은 어깨의 유착성 관절낭염으로 어깨의 관절낭에 염증이 생기거나, 이와 동반되어 통증 및 굳어지는 증상이 생기는 질환입니다. 오십견이라 불리지만 반드시 오십대나 나이든 사람에게만 생기는 것은 아니며, 그보다 젊은 연령층에서도 발생할 수 있습니다. 팔을 많이 쓰는 사람이나 주부에게 많이 오는 것이 특징이며 최근에는 잘못된 자세로 컴퓨터를 장시간 사용하거나 스트레스, 운동부족 등도 원인이 되는 것으로 알려져 있습니다.', metada

chain_type = 'stuff'의 내부처리

```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

{텍스트}에는 입력데이터와 유사도가 높은 텍스트의 본문이 삽입되고,  
{질문}에는 입력 텍스트가 삽입되는 구조이다.

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0),
    chain_type = 'stuff',
    retriever = retriever,
    return_source_documents = True)

### Query

In [17]:
input_text = '오십견 알려줘.'
chatbot_response = qa_chain(input_text)
print(chatbot_response)

{'query': '오십견 알려줘.', 'result': '오십견은 어깨의 유착성 관절낭염으로 어깨의 관절낭에 염증이 생기거나, 이와 동반되어 통증 및 굳어지는 증상이 생기는 질환입니다. 오십견이라 불리지만 반드시 오십대나 나이든 사람에게만 생기는 것은 아니며, 그보다 젊은 연령층에서도 발생할 수 있습니다. 팔을 많이 쓰는 사람이나 주부에게 많이 오는 것이 특징이며 최근에는 잘못된 자세로 컴퓨터를 장시간 사용하거나 스트레스, 운동부족 등도 원인이 되는 것으로 알려져 있습니다.', 'source_documents': [Document(page_content='오십견은 나이든 사람에게만 생기나요 ?\n아닙니다. 흔히 어깨가 아프면 오십견이라고 생각하기 쉬우나 어깨 통증의 원인은 오십견 뿐만 아니라 다음과 같이 많은 질환에 의해 야기될 수 있습니다.\n<어깨통증의 원인>\n1.오십견 (유착성 견관절낭염) 2.어깨 관절 충돌 증후군 (건초염, 점액낭염)\n3.회전근개 파열\n4.근막통 증후군\n5.관절염\n6.경추 디스크 질환\n7.경추 퇴행성 질환\n8.경추 신경근 병변\n9.내장 질환 (심근경색증, 폐 질환 등 )\n그중 오십견은 어깨관절의 한 구조물인 피낭의 염증으로 피낭을 섬유화하고 두껍게 하며, 활액의 반응도 유착되어 정상적인 피낭의 유연성이 소실되고, 외회전, 회전 및 내전의 가동력을 제한받는 것을 말합니다. 오십견은 다음과 같은 경우 발생될 수 있습니다.\n<오십견의 원인>\n1. 움직이지 않아서 발생\n2. 자가 면역 반응\n3. 부상 후 발생\n4. 점액낭염, 건초염, 충돌 증후군, 회전근개 파열과 동반\n이렇게 어깨구조물의 염증이 야기되고 최종적으로 어깨관절이 굳은 현상을 유착성 낭염이라고 합니다. 이것은 보통 오십대에 많이 발생해서 오십견으로 흔히 알려져 왔습니다. 그러나 오십견은 위와같은 원인이 있는 경우 연령에 상관없이 누구에게나 발생할 수 있습니다. 최근에는 운동부족과 장시간에 걸친 어깨의 과도한 사용으로 20-30대에서도 흔히 발생됩니다.\

In [18]:
def get_chatbot_response(chatbot_response):
  print(chatbot_response['result'].strip())
  print('\n문서 출처:')
  for source in chatbot_response['source_documents']:
    print(source.metadata['source'])

In [19]:
input_text = '다이어트 어떻게 해야해?'
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

다이어트를 위해서는 몇 가지 중요한 요소를 고려해야 합니다. 

1. 식단 조절: 칼로리 섭취를 줄이기 위해 과일, 채소, 단백질이 풍부한 식품을 섭취하고, 과다한 포만감을 주는 음식이나 당분이 많은 음식은 피해야 합니다. 또한, 식사를 규칙적으로 하고 과식을 피하는 것이 중요합니다.

2. 운동: 유산소 운동과 근력 운동을 조합하여 신체 활동량을 늘리고 근육을 강화하는 것이 좋습니다. 유산소 운동으로는 걷기, 달리기, 수영 등이 있으며, 근력 운동으로는 체중 트레이닝, 요가, 필라테스 등을 선택할 수 있습니다.

3. 건강한 생활습관: 충분한 수면을 취하고 스트레스를 관리하는 것이 중요합니다. 또한, 알코올과 담배를 줄이거나 피하는 것도 좋습니다.

4. 전문가와의 상담: 다이어트에 대한 전문가의 조언을 듣고 개인에게 맞는 목표와 방법을 설정하는 것이 좋습니다. 전문가와의 상담을 통해 식단과 운동 계획을 세울 수 있습니다.

다이어트를 시작하기 전에 건강 상태와 목표를 고려하여 계획을 세우는 것이 중요합니다. 무리한 다이어트는 건강에 해로울 수 있으므로 꾸준하고 안전한 방법으로 다이어트를 진행하는 것이 좋습니다.

문서 출처:
data/data_hwell/44.txt
data/data_hwell/3.txt


In [20]:
chatbot_response

{'query': '다이어트 어떻게 해야해?',
 'result': '다이어트를 위해서는 몇 가지 중요한 요소를 고려해야 합니다. \n\n1. 식단 조절: 칼로리 섭취를 줄이기 위해 과일, 채소, 단백질이 풍부한 식품을 섭취하고, 과다한 포만감을 주는 음식이나 당분이 많은 음식은 피해야 합니다. 또한, 식사를 규칙적으로 하고 과식을 피하는 것이 중요합니다.\n\n2. 운동: 유산소 운동과 근력 운동을 조합하여 신체 활동량을 늘리고 근육을 강화하는 것이 좋습니다. 유산소 운동으로는 걷기, 달리기, 수영 등이 있으며, 근력 운동으로는 체중 트레이닝, 요가, 필라테스 등을 선택할 수 있습니다.\n\n3. 건강한 생활습관: 충분한 수면을 취하고 스트레스를 관리하는 것이 중요합니다. 또한, 알코올과 담배를 줄이거나 피하는 것도 좋습니다.\n\n4. 전문가와의 상담: 다이어트에 대한 전문가의 조언을 듣고 개인에게 맞는 목표와 방법을 설정하는 것이 좋습니다. 전문가와의 상담을 통해 식단과 운동 계획을 세울 수 있습니다.\n\n다이어트를 시작하기 전에 건강 상태와 목표를 고려하여 계획을 세우는 것이 중요합니다. 무리한 다이어트는 건강에 해로울 수 있으므로 꾸준하고 안전한 방법으로 다이어트를 진행하는 것이 좋습니다.',
 'source_documents': [Document(page_content='남성에 있어서도 비만이 혈관질환과 내분비장애를 초래하여 성욕의 저하, 발기 부전, 불임등을 초래하는 것으로 알려져 있습니다.\n그러므로, 체중감량을 하게 되면 남성이나 여성 모두 성기능에 긍정적인 영향을 미친다고 할수 있겠습니다.\n주의해야 할 점은 무리한 다이어트를 하게 되면 성기능에 부정적인 영향을 미칠 수도 있다는 것입니다. 여성의 경우 무리한 다이어트로 인해 단백질이 심하게 부족하게 되면 성호르몬에 영향을 받게 되어 생식능력에 장애가 올수 있습니다. 또한 생명 유지에 꼭 필요하지 않은 기능부터 손상되기 때문에 생리 불순에 걸리거나 심할 때는 아예 생리가 없어지기도 합니다. 

In [22]:
input_text = '다이어트 어떻게 해야해?'
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

다이어트를 위해서는 몇 가지 중요한 점을 고려해야 합니다. 

1. 식단 조절: 칼로리 섭취를 줄이기 위해 과일, 채소, 단백질이 풍부한 식품을 섭취하고, 과다한 지방과 당을 피해야 합니다. 균형 잡힌 식단을 유지하며, 식사를 규칙적으로 하고 과식을 피해야 합니다.

2. 운동: 유산소 운동과 근력 운동을 조합하여 신체 활동량을 늘리고 근육을 강화해야 합니다. 걷기, 달리기, 수영 등의 유산소 운동과 체력 단련을 위한 근력 운동을 선택할 수 있습니다.

3. 생활 습관: 충분한 수면을 취하고 스트레스를 관리하는 것이 중요합니다. 스트레스로 인해 과식을 하는 경우가 많으므로, 스트레스를 효과적으로 해소하는 방법을 찾아야 합니다.

4. 전문가 상담: 다이어트에 대한 전문가와의 상담을 통해 개인에게 맞는 목표와 방법을 설정하는 것이 좋습니다. 전문가의 조언과 도움을 받으면 더 효과적인 다이어트를 할 수 있습니다.

다이어트는 각 개인의 몸 상태와 목표에 따라 다를 수 있으므로, 자신에게 맞는 방법을 찾아 실천하는 것이 중요합니다.

문서 출처:
data/data_hwell/44.txt
data/data_hwell/3.txt


In [23]:
query = '다이어트 어떻게 해야해?'
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

다이어트를 위해서는 몇 가지 중요한 요소를 고려해야 합니다. 

1. 식단 조절: 칼로리 섭취를 줄이기 위해 과일, 채소, 단백질이 풍부한 식품을 섭취하고, 과다한 포만감을 주는 음식이나 당분이 많은 음식은 피해야 합니다. 또한, 식사를 규칙적으로 하고 과식을 피하는 것이 중요합니다.

2. 운동: 유산소 운동과 근력 운동을 조합하여 신체 활동량을 늘리고 근육을 강화하는 것이 좋습니다. 유산소 운동으로는 걷기, 달리기, 수영 등이 있으며, 근력 운동으로는 체중 트레이닝, 요가, 필라테스 등을 선택할 수 있습니다.

3. 건강한 생활습관: 충분한 수면을 취하고 스트레스를 관리하는 것이 중요합니다. 또한, 알코올과 담배를 줄이거나 피하는 것도 좋습니다.

4. 전문가와의 상담: 개인의 몸 상태와 목표에 맞는 다이어트 방법을 위해 전문가와 상담하는 것이 좋습니다. 영양사나 트레이너와 함께 목표를 설정하고 계획을 세울 수 있습니다.

다이어트는 각 개인의 몸 상태와 목표에 따라 다를 수 있으므로, 자신에게 맞는 방법을 찾기 위해 전문가와의 상담이 필요합니다.

문서 출처:
data/data_hwell/44.txt
data/data_hwell/3.txt


### Gradio로 챗봇 UI만들기

In [25]:
!pip install gradio==3.40.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 5.9 MB/s eta 0:00:0

To create a public link, set

In [26]:
import gradio as gr

# 인터페이스 생성
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(label='건강FAQ챗봇') # 레이블 좌측 상단에 구성
  msg = gr.Textbox(label='질문해주세요!') # 레이블 하단에 구성
  clear = gr.Button('대화 초기화') # 초기화 버튼

  # 챗봇 답변 처리하는 함수
  def respond(message, chat_history):
    result = qa_chain(message)
    bot_message = result['result']
    bot_message += '# sources:'

    # 답변 출처 표기
    for i, doc in enumerate(result['source_documents']):
      bot_message += '['+str(i+1)+']' + doc.metadata['source'] + ' '

    # 채팅 기록에 사용자의 메세지와 봇의 응답을 추가
    chat_history.append((message, bot_message))
    return '', chat_history

  # 사용자의 입력을 제출하면 respond 함수가 호출
  msg.submit(respond, [msg, chatbot], [msg, chatbot])

  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
  clear.click(lambda: None, None, chatbot, queue = False)


demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
